In [12]:
# conda remove --name SCENIC --all
# conda create --name SCENIC python=3.6.3 
# conda activate SCENIC
# python -m pip install pandas
# git clone https://github.com/aertslab/pycisTopic.git
# cd pycisTopic
# python -m pip install .
# python -m pip install --upgrade pip setuptools
# python -m pip install scanpy
# python -m pip install  pyranges
# python -m pip install requests --ignore-installed certifi
# python -m pip install ray
# python -m pip install tmtoolkit
# python -m pip install gensim
# python -m pip install loomxpy
# python -m pip install pyrle
# python -m pip install polars
# python -m pip  install typing-extensions --upgrade
# python -m pip install pyarrow
# python -m pip install loompy
# python -m pip install pyscenic
# python -m pip install pickle5
# python -m pip install ipykernel
#  python -m pip install pyBigWig
# install jupyter-lab
# mv pycisTopic/ pycisTopic_repo
#git clone https://github.com/aertslab/scenicplus
#cd scenicplus
#pip install .



import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
import scanpy
import os
import requests
import pyranges as pr
import pycisTopic
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
from pycisTopic.iterative_peak_calling import *
import anndata
import pickle
import pyarrow 
import re
from pycisTopic.lda_models import *
from pycisTopic.cistopic_class import *
import scrublet as scr
from pycisTopic.lda_models import *
from pycisTopic.clust_vis import *



def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for items in my_list:
        freq[items] = my_list.count(items)
 
    for key, value in freq.items():
        print("% s : % d" % (key, value))
 

outDir="SCENIC_results_MARCH2024/"
tmpDir="/tmp/"
#ray.shutdown()

In [13]:
genomeref = pd.read_pickle(r"~/tasks/margo.montandon/Analysis/GenomeRef_pyAnnotation.obj")
genomeref

,Chromosome,Start,End,Name
0,1,0,43039672,1
1,2,0,39555667,2
2,3,0,60282382,3
3,4,0,76270779,4
4,5,0,61329656,5
...,...,...,...,...
262,KZ116053.1,0,10784,KZ116053.1
263,KZ116055.1,0,1335,KZ116055.1
264,KZ116064.1,0,82859,KZ116064.1
265,KZ116065.1,0,20619,KZ116065.1


In [14]:
import pickle
infile = open(outDir + 'cisTopicObject_noDBL.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()
# Load imputed accessibility
import pickle
infile = open(outDir + 'DARs/Imputed_accessibility.pkl', 'rb')
imputed_acc_obj = pickle.load(infile)
infile.close()
# Load DARs
import pickle
infile = open(outDir + 'DARs/DARs.pkl', 'rb')
DARs_dict = pickle.load(infile)
infile.close()

In [1]:
import pybiomart as pbm
dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://www.ensembl.org')
#dataset
#dataset = pbm.Dataset(name='drerio_gene_ensembl',  host='http://asia.ensembl.org')
dataset

<biomart.Dataset name='drerio_gene_ensembl', display_name=''>

In [4]:
annot = dataset.query(attributes=['chromosome_name', 'start_position', 'end_position', 'strand', 'external_gene_name', 'transcription_start_site', 'transcript_biotype'])
annot

,<html>
0,<head>
1,<title>Service unavailable</title>
2,"<meta name=""robots"" content=""noindex"">"
3,"<base href=""//status.ensembl.org"">"
4,<style>
...,...
75,<footer>
76,<p>&copy; 2024 EMBL-EBI</p>
77,</footer>
78,</body>


In [3]:
annot.columns=['Chromosome', 'Start', 'End', 'Strand', 'Gene','Transcription_Start_Site', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']
annot.Strand[annot.Strand == 1] = '+'
annot.Strand[annot.Strand == -1] = '-'
pr_annotation = pr.PyRanges(annot.dropna(axis = 0))


ValueError: Length mismatch: Expected axis has 1 elements, new values have 7 elements

In [ ]:
pr_annotation

NameError: name 'pr_annotation' is not defined

In [ ]:
from pycisTopic.gene_activity import *
gene_act, weigths = get_gene_activity(imputed_acc_obj, # Region-cell probabilities
                pr_annotation, # Gene annotation
                 genomeref, # Chromosome size
                use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
                upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it
                                      #these bp will be taken (1kbp here)
                downstream=[1000,100000], # Search space downstream
                distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
                decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
                extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for
                                      #this weight)
                extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
                gene_size_weight=False, # Whether to add a weights based on the length of the gene
                gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                                      #in the genome
                remove_promoters=False, # Whether to remove promoters when computing gene activity scores
                average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene
                                      #activity score
                scale_factor=1, # Value to multiply for the final gene activity matrix
                extend_tss=[10,10], # Space to consider a promoter
                gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
                return_weights= True, # Whether to return the final weights
                project='Gene_activity') # Project name fo

In [ ]:
# your ATAC reads are "expression"

markers_dict= find_diff_features(cistopic_obj,
                      gene_act,
                      variable='clusters',
                      var_features=None,
                      contrasts=None,
                      adjpval_thr=0.05,
                      log2fc_thr=np.log2(1.5),
                      n_cpu=5,
                      _temp_dir=tmpDir + 'ray_spill',
                      split_pattern = '-')

In [ ]:
first_value = next(iter(markers_dict.values()))
first_value.index

In [ ]:
os.mkdir(outDir+'DAGs')
# is the gene accessible in that cluster?
from pycisTopic.clust_vis import *
plot_imputed_features(cistopic_obj,
                    reduction_name='Seurat_RNA+ATAC_UMAP',
                    imputed_data=gene_act,
                    features=first_value.index[1:13], #Glia
                    scale=True,
                    num_columns=4,
                    save= outDir + 'DAGs/example_best_DAGs.pdf')

In [ ]:
x = [print(x + ': '+ str(len(markers_dict[x]))) for x in markers_dict.keys()]


In [ ]:
# Save
with open(outDir + 'DAGs/Gene_activity.pkl', 'wb') as f:
  pickle.dump(gene_act, f)
with open(outDir + 'DAGs/DAGs.pkl', 'wb') as f:
  pickle.dump(markers_dict, f)

In [ ]:


# gene_activity file. no loom
import pickle
infile = open(outDir + 'DAGs/Gene_activity.pkl', 'rb') #Here I am using pycisTopic gene activity matrix, but could be any :)
gene_act = pickle.load(infile)
infile.close()

# Load cisTopic object
import pickle
infile = open(outDir + 'cisTopicObject_noDBL.pkl', 'rb')
cistopic_obj = pickle.load(infile)
infile.close()
# Load imputed accessibility
import pickle
infile = open(outDir + 'DARs/Imputed_accessibility.pkl', 'rb') #Here I am using pycisTopic gene activity matrix, but could be any
imputed_acc_obj = pickle.load(infile)
infile.close()
# Load region binarized topics
import pickle
infile = open(outDir + 'topic_binarization/binarized_topic_region.pkl', 'rb')
binarized_topic_region = pickle.load(infile)
infile.close()
# Load cell binarized topics
import pickle
infile = open(outDir + 'topic_binarization/binarized_cell_topic.pkl', 'rb')
binarized_cell_topic = pickle.load(infile)
infile.close()
# Load DARs
import pickle
infile = open(outDir + 'DARs/DARs.pkl', 'rb')
DARs_dict = pickle.load(infile)
infile.close()

